# 비수기 linear regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
#데이터 로드
train_data = pd.read_csv(r"data45910_train.csv", encoding='cp949')
test_data = pd.read_csv(r"data45910_test.csv", encoding='UTF-8')

In [3]:
#데이터 전처리
train_data['모기 개체수'] = train_data['모기 개체수'].str.replace(',', '').astype(int)
test_data['모기 개체수'] = test_data['모기 개체수'].str.replace(',', '').astype(int)

In [4]:
features = [
    '평균기온', '최저기온', '최고기온', '일강수량(mm)', '최대 풍속(m/s)', '평균 풍속(m/s)',
    '최소 상대습도(%)', '평균 상대습도(%)', '평균 현지기압(hPa)', '평균 해면기압(hPa)',
    '합계 일조시간(hr)', '합계 일사량(MJ/m2)', '평균 전운량(1/10)', '평균 지면온도','1일전 모기 개체수','2일전 모기 개체수','3일전 모기 개체수']

target = '모기 개체수'

In [5]:
#데이터 split
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [6]:
#데이터 scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [8]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [9]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 1504649.5418
Train RMSE : 1226.6416
Train R^2 : 0.6148
Performance for TEST--------
Test MSE : 180419.4195
Test RMSE : 424.7581
Test R^2 : 0.6237


# 비수기 Ridge

In [10]:
from sklearn.linear_model import Ridge

In [11]:
#hyperparameter를 정하기 위한 validation set 분리
X_split_train, X_split_val, y_split_train, y_split_val = train_test_split(X_train_scaled, y_train, random_state=42)  

In [12]:
alphas = [0.01, 0.1, 1, 10, 100, 1000]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Ridge(alpha=alpha)
    model.fit(X_split_train, y_split_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_split_train)
    y_val_hat = model.predict(X_split_val)

    # R^2 값 계산
    train_r2 = r2_score(y_split_train, y_train_hat)
    val_r2 = r2_score(y_split_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')

Alpha: 0.01
Train R^2: 0.7179
Validation R^2: 0.4193
---------------------
Alpha: 0.1
Train R^2: 0.7177
Validation R^2: 0.4208
---------------------
Alpha: 1
Train R^2: 0.7108
Validation R^2: 0.4230
---------------------
Alpha: 10
Train R^2: 0.6150
Validation R^2: 0.3632
---------------------
Alpha: 100
Train R^2: 0.3148
Validation R^2: 0.1729
---------------------
Alpha: 1000
Train R^2: 0.0624
Validation R^2: 0.0234
---------------------


In [13]:
model = Ridge(alpha=0.1) #alpha = 0.1 채택
model.fit(X_train_scaled, y_train)

Ridge(alpha=0.1)

In [14]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [15]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 1505006.4868
Train RMSE : 1226.7871
Train R^2 : 0.6148
Performance for TEST--------
Test MSE : 183263.0811
Test RMSE : 428.0924
Test R^2 : 0.6178


# 비수기 Lasso

In [16]:
from sklearn.linear_model import Lasso

In [17]:
alphas = [0.01, 0.1, 1, 10, 100]

train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    model = Lasso(alpha=alpha)  
    model.fit(X_split_train, y_split_train)

    y_train_hat = model.predict(X_split_train)
    y_val_hat = model.predict(X_split_val)

    train_r2 = r2_score(y_split_train, y_train_hat)
    val_r2 = r2_score(y_split_val, y_val_hat)
    
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"validation R^2: {val_r2:.4f}")
    print('---------------------')

Alpha: 0.01
Train R^2: 0.7179
validation R^2: 0.4191
---------------------
Alpha: 0.1
Train R^2: 0.7179
validation R^2: 0.4189
---------------------
Alpha: 1
Train R^2: 0.7174
validation R^2: 0.4161
---------------------
Alpha: 10
Train R^2: 0.7100
validation R^2: 0.3933
---------------------
Alpha: 100
Train R^2: 0.4329
validation R^2: 0.1943
---------------------


In [18]:
model = Lasso(alpha=0.01) #0.01채택
model.fit(X_train_scaled, y_train)

Lasso(alpha=0.01)

In [19]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [20]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 1504792.2273
Train RMSE : 1226.6997
Train R^2 : 0.6148
Performance for TEST--------
Test MSE : 180341.3468
Test RMSE : 424.6662
Test R^2 : 0.6239
